In [ ]:
from __future__ import print_function
import pandas as pd
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import GRU
from keras.layers import CuDNNLSTM
from keras.layers import BatchNormalization
from keras.layers import TimeDistributed
from keras.layers import Bidirectional

import math
import os
import time


from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np
import random
import sys
import io
import music21
import datetime
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
pip install guitarpro



In [ ]:
# load note coding labels 
labels = pd.read_csv('output/labelled_notes.csv')
labels = labels.values.tolist()

print('total labels:', len(labels))

In [ ]:
df = pd.read_csv('output/processed_output.csv')
df.drop(columns=['Unnamed: 0'], inplace=True)

df

In [ ]:
#def process_row(row):

maxlen = 32
step = 1
dim = len(labels)
#df = df[df['song'] == 1]

def old_create_batches():
    sequences = np.zeros(shape=(len(df), maxlen, dim), dtype=np.int8)
    next_notes = np.zeros(shape=(len(df), dim), dtype=np.int8)

    songs_count = len(df['song'].unique())

    batch_id = 0
    for song_idx in range(1, songs_count):
        df_part = df[df['song'] == song_idx]
        df_part.drop(columns=['song'], inplace=True)
        length = len(df_part) - maxlen - 1
    
        for slice_start in range(0, length, step):
            sample_id = 0
            for row in df_part[slice_start: maxlen + slice_start].iterrows():
            
                sequences[batch_id][sample_id] = row[1]
            
                sample_id += 1
            next_notes[batch_id] = row[1]
            batch_id += 1
            
    return sequences, next_notes, songs_count

            
def create_batches(): #94% better perf
    sequences = np.zeros(shape=(len(df), maxlen, dim), dtype=np.int8)
    next_notes = np.zeros(shape=(len(df), dim), dtype=np.int8)

    songs_count = len(df['song'].unique())

    batch_id = 0
    for song_idx in range(1, songs_count):
        df_part = df[df['song'] == song_idx]
        df_part.drop(columns=['song'], inplace=True)
        bucket_count = len(df_part) - maxlen - 1
    
        for slice_start in range(0, bucket_count, step):

            next_notes[batch_id] = df_part.iloc[maxlen + slice_start + 1]
            sequences[batch_id] = df_part[slice_start: maxlen + slice_start].to_numpy()
            batch_id += 1
            
    return sequences, next_notes, songs_count



t1 = time.time()
sequences, next_notes, songs_count = create_batches()

t2 = time.time()
print (t2 - t1)  

#t1 = time.time()
#sequences, next_notes, songs_count = old_create_batches()

#t2 = time.time()
#print (t2 - t1)  

print('nb songs: ', songs_count)
print('nb sequences:', len(sequences))
print('nb next_notes:', len(next_notes))

In [ ]:
import tensorflow_addons as tfa
labels_count = len(labels)

model = Sequential()
model.add(LSTM(512, return_sequences=True, input_shape=(maxlen, labels_count)))
model.add(Dropout(0.2))
model.add(LSTM(512, return_sequences=True, activation=tfa.activations.gelu))
model.add(Dropout(0.3))
model.add(LSTM(512, return_sequences=False, activation=tfa.activations.gelu))
model.add(Dense(512, activation=tfa.activations.gelu))
model.add(Dropout(0.3))
model.add(Dense(labels_count, activation='softmax'))

 
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc', 'categorical_crossentropy'])

In [ ]:
#model 2
import tensorflow_addons as tfa
labels_count = len(labels)

model = Sequential()
model.add(LSTM(512, return_sequences=True, input_shape=(maxlen, labels_count)))
model.add(Dropout(0.3))
model.add(Bidirectional(LSTM(512, return_sequences=True, activation="relu")))
model.add(Dropout(0.3))
model.add(LSTM(512, return_sequences=False, activation="relu"))
model.add(Dense(512, activation="relu"))
model.add(Dropout(0.3))
model.add(Dense(labels_count, activation='softmax'))

 
model.compile(loss='categorical_crossentropy', optimizer='adam')
print('model 2')

In [ ]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

def decode(index):
    return labels[index][0]

def sequence_to_label(sequence):
    result = []
    for index, data in sequence.iterrows():
        for label, content in data.items():
            if content > 0 :
                result.append(label)
    return result

def export_to_midi(string, name):
    notes = string.strip().split(' ')
    s = music21.stream.Stream()
    for row in notes:
        data = row.split("-")
        note = data[0]
        duration= float(data[1])
    
        if not note == 'x':
            if note == 'r':
                s.append(music21.note.Rest(quarterLength=duration))
            else:
                s.append(music21.note.Note(note, quarterLength=duration))

    print(s)
    s.write("midi", 'predicted/' + name)
    
def get_file_name(epoch, diversity):
    now = datetime.datetime.now()
    return now.strftime("%Y%m%d%H%M") + '_' +str(epoch) +'_' + str(diversity) + '.mid'
    
def get_seed():
    return df_copy[start_index: start_index + maxlen]

def on_epoch_end(epoch, _):
    # Function invoked at end of each epoch. Prints generated text.
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(df) - maxlen - 1)
    start_index = 128
    for diversity in [0.2,0.3,0.4,0.5,0.6]:
        print('----- diversity:', diversity)
        
        df_copy = df.drop(columns=['song'])
        sequence = df_copy[start_index: start_index + maxlen]
        
        #print('----- Generating with seed: "' + ','.join(sequence_to_label(sequence)) + '"') 
        #print()
        predicted = ''
        for i in range(1024):
            # czary mary
            sequences = np.zeros(shape=(1, maxlen, len(labels)), dtype=float)
            np.append(sequences, sequence)
            x_pred = sequences

            # predicting 
            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            
            
            next_char = decode(next_index)

            #adding new note TODO
            new_note = np.zeros(shape=(len(labels)), dtype=float)
            new_note[next_index] = 1
            
            sequence = sequence[1:]
            sequence.append(pd.DataFrame(new_note), ignore_index=True)
            
            #add
            predicted += next_char + ' '
            #writing
            sys.stdout.write(next_char + ' ')
            sys.stdout.flush()
            
        export_to_midi(predicted, get_file_name(epoch, diversity))
        print()

print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

history = model.fit(sequences, next_notes,
          batch_size=128,
          epochs=20,
          callbacks=[print_callback])

In [ ]:
for label in ["acc"]:
    plt.plot(history.history[label],label=label)
plt.legend()
plt.show()

for label in ["loss"]:
    plt.plot(history.history[label],label=label)
plt.legend()
plt.show()

In [ ]:
# export to midi file

import music21
string = 'd2-0.5 d2-0.5 d2-0.5 d2-0.5 d2-0.5 d3-0.25 d2-0.5 a2-0.5 d2-0.5 d2-0.5 d2-0.5 r-0.5 d2-0.5 d2-0.5 d2-0.5 a2-0.5 d3-0.5 d3-0.5 a2-0.5 d2-0.5 d3-0.5 d2-0.5 d2-0.5 d2-0.3333333333333333 d2-0.5 d2-0.5 d2-0.5 b2-0.5 d2-0.5 d2-0.5 d2-0.5 e2-0.25 d3-0.5 d2-0.5 d2-0.5 d2-0.5 d2-0.5 d3-0.5 a2-0.5 a2-0.5 d2-0.5 r-0.5 d2-0.5 d2-0.5 d2-0.5 d2-0.5 d2-0.5 d3-0.5 r-0.5 d2-0.5 d2-0.5 d2-0.5 d3-0.5 d2-0.5 d2-0.5 d2-0.5 d2-0.3333333333333333 d2-0.5 d2-0.5 d2-0.5 d2-0.5 d2-0.3333333333333333 d2-0.5 d2-0.5 d2-0.5 b2-0.5 d2-0.3333333333333333 e3-0.25 a2-0.5 d2-0.5 d2-0.5 d2-0.3333333333333333 d2-0.3333333333333333 d2-0.5 d2-0.5 d3-0.5 d3-0.5 d2-0.5 d2-0.5 d3-0.5 d2-0.5 d2-0.5 d2-0.5 d2-0.5 d2-0.5 b2-0.5 d2-0.5 d3-0.5 d2-0.5 d2-0.5 d2-0.5 d2-0.5 d2-0.3333333333333333 d2-0.5 e2-0.5 d2-0.5 d2-0.5 d2-0.5 d2-0.5 d2-0.5 d2-0.5 d2-0.3333333333333333 d2-0.5 a2-1.0 d2-0.3333333333333333 d2-0.5 d3-1.0 d2-0.5 d2-0.5 d2-0.5 d2-0.5 d2-0.5 a2-0.5 d2-0.5 d3-0.5 a2-0.5 d2-0.5 d2-0.3333333333333333 d2-0.5 d3-0.5 d2-0.5 d2-0.5 d2-0.5 d2-0.5 e3-0.5 d2-0.5 d2-0.5 d2-0.5 d2-0.5 d2-0.5 d3-0.5 d2-0.5 d2-0.5 e3-0.25 d2-0.5 d2-0.3333333333333333 d3-0.5 a2-0.5 d2-0.5 d3-0.5 d2-0.5 d2-0.5 d2-0.5 d2-0.5 d2-0.5 d2-0.5 b2-0.5 d3-0.5 d2-0.5 d2-0.5 e2-0.5 d2-0.5 e2-0.5 a2-0.5 a2-0.5 e2-0.5 d2-0.5 d2-0.5 d2-0.5 d2-0.5 d2-0.5 d2-0.5 d2-0.5 e3-0.5 d2-0.5 d2-0.5 d3-0.5 d2-0.5 d3-0.5 d3-0.6666666666666666 d2-0.5 d2-0.5 d2-0.5 d2-0.5 d2-0.5 d2-0.5 d2-0.5 d2-0.5 a2-0.5 d2-0.5 d3-0.25 d2-0.5 d2-0.5 d2-0.5 r-0.5 d2-0.5 d2-0.5 d2-0.5 d2-0.5 e2-0.5 d3-0.5 d2-0.5 d2-0.5 d2-0.5 d2-0.5 d3-0.5 a2-0.5 d2-0.5 d2-0.5 d2-0.5 d2-0.3333333333333333 d2-0.5 d2-0.3333333333333333 g3-0.5 d2-0.5 c3-0.3333333333333333 d2-0.5 d2-0.5 d2-0.5 d2-0.5 d2-0.5 b2-0.5 a2-0.5 d2-0.5 d2-0.5 d3-0.5 d2-0.5 d2-0.5 d2-0.5 d2-0.5 d2-0.5 d2-0.5 d2-0.5 d2-0.5 d2-0.5 a2-0.5 d3-0.25 d2-0.5 d2-0.5 d3-0.5 d2-0.5 a2-0.5 d3-0.5 a2-0.5 e2-0.5 d2-0.5 d2-0.5 d2-0.5 d2-0.5 d3-0.25 d2-0.5 d2-0.5 a2-0.5 d2-0.3333333333333333 d2-0.5 d2-0.5 d2-0.5 a2-0.25 d2-0.5 d2-0.5 a2-0.5 d3-0.5 d2-0.5 d2-0.5 d2-0.5 d2-0.5 d2-0.5 d2-0.5 d2-0.3333333333333333 d2-0.5 d2-0.5 d2-0.5 d2-0.5 e3-0.5 e2-0.5 d2-0.5 d2-0.5 d3-0.5 d2-0.3333333333333333 d2-0.5 e2-0.5 c#3-0.25 d2-0.5 d2-0.5 a2-0.5 d2-0.5 d2-0.5 d2-0.5 d2-0.5 d2-0.5 d2-0.5 d2-0.5 d2-0.5 d2-0.5 d2-0.5 d2-0.5 d2-0.5 a2-0.5 d2-0.5 a2-0.5 d2-0.5 d2-0.3333333333333333 d2-0.5 d2-0.5 d2-0.5 d2-0.5 d3-0.25 a2-0.5 d2-0.5 d2-0.3333333333333333 d2-0.5 d2-0.5 e3-0.5 d2-0.5 d2-0.5 e3-0.5 e3-0.5 d2-0.5 d2-0.5 d2-0.5 d2-0.5 d2-0.5 d2-0.5 d2-0.5 e2-0.5 a2-0.5 d2-0.5 d2-0.5 d2-0.5 d2-0.5 b2-0.5 d2-0.5 e2-0.5 d2-0.5 d2-0.5 d2-0.3333333333333333 d2-0.25 d2-0.5 a2-0.5 d2-0.3333333333333333 d3-0.5 d2-0.5 d2-0.5 d2-0.5 d2-0.5 e2-0.5 d3-0.5 d2-0.5 d2-0.3333333333333333 d2-0.5 d2-0.5 d2-0.5 e3-0.5 d2-0.3333333333333333 a2-0.5 d2-0.5 d2-0.5 d2-0.5 d3-0.25 d2-0.5 d2-0.5 d2-0.5 d2-0.3333333333333333 d2-0.5 a2-0.5 a2-0.5 d2-0.5 d2-0.5 d3-0.5 a2-0.5 d2-0.5 d2-0.5 d2-0.3333333333333333 d2-0.5 d2-0.5 d2-0.5 d2-0.5 d2-0.3333333333333333 r-0.5 a2-0.5 d2-0.5 d3-0.5 d2-0.3333333333333333 d2-0.5 d2-0.5 d2-0.5 a2-0.5 d3-0.5 d2-0.5 d2-0.5 a2-0.5 d2-0.5 d2-0.5 d2-0.5 d2-0.5 d2-0.5 d2-0.5 d2-0.5 d2-0.5 e3-0.5 d2-0.5 d2-0.3333333333333333 d2-0.3333333333333333 d3-0.5 d2-0.5 d2-0.5 d2-0.5 a2-0.5 d3-0.5 a2-0.5 d2-0.5 d2-0.5 d2-0.5 d2-0.5 d2-0.5 d2-0.5 b2-0.5 a2-0.5 d2-0.5 d2-0.5 d2-0.5 d2-0.5 d2-0.5 d2-0.5 e3-0.5 d2-0.5 d3-0.5 d2-0.5 d2-0.5 d3-0.5 d2-0.5 d2-0.5 d2-0.3333333333333333 d2-0.5 d3-0.5 d2-0.5 a2-0.5 d2-0.5 d2-0.5 a2-0.5 d2-0.5 d2-0.5 a2-0.5 d2-0.5 d2-0.5 d3-0.5 d2-0.5 d2-0.5 a2-0.5 a2-0.5 a2-0.5 a2-0.5 d3-0.5 d3-0.5 a2-0.5 d2-0.5 d2-0.5 d2-0.5 d2-0.5 d3-0.5 d2-0.5 d3-0.5 d2-0.5 d2-0.5 d2-0.5 d2-0.5 a2-0.5 d2-0.5 d2-0.5 d2-0.5 d2-0.5 d2-0.5 e2-0.5 d2-0.5 d2-0.5 d2-0.5 d2-0.5 d2-0.3333333333333333 d2-0.5 d2-0.5 d2-0.5 d2-0.5 d2-0.5 d3-0.5 d2-0.3333333333333333 d3-0.5 d2-0.5 d2-0.5 d2-0.5 d2-0.5 d2-0.5 d2-0.5 a2-0.5 d2-0.5 d2-0.5 d2-0.3333333333333333 a2-0.5 d2-0.3333333333333333 d2-0.3333333333333333 a2-0.5 d2-0.5 d2-0.5 d2-0.5 r-0.5 d2-0.5 d2-0.5 d2-0.5 d2-0.5 d2-0.3333333333333333 e2-0.5 a2-0.5 d2-0.5 e3-0.5 d2-0.5 d2-0.5 a2-0.5 d3-0.5 d2-0.5 d2-0.5 d3-0.5 a2-0.25 d2-0.5 d2-0.5 d2-0.5 a2-0.5 d2-0.5 d2-0.5 d2-0.5 d2-0.5 d2-0.25 d2-0.3333333333333333 d2-0.5 d2-0.5 d2-0.5 d2-0.5 d2-0.5 a2-0.5 d2-0.5 d2-0.5 d2-0.5 d2-0.5 d2-0.3333333333333333 d3-0.5 d2-0.5 d2-0.5 d2-0.5 d2-0.5 d2-0.5 d2-0.5 d2-0.5 d2-0.3333333333333333 d2-0.5 d2-0.5 d2-0.5 d2-0.5 d2-0.5 a2-0.5 d2-0.5 d2-0.5 d2-0.5 a2-0.5 d2-0.5 d2-0.3333333333333333 d2-0.3333333333333333 a2-0.25 d2-0.5 d2-0.5 d2-0.3333333333333333 d2-0.5 d2-0.5 d2-0.5 d2-0.5 d3-0.25 d2-0.5 d2-0.5 d2-0.5 d2-0.5 a2-0.5 d2-0.5 a2-0.5 a2-0.5 d2-0.5 a2-0.5 d2-0.3333333333333333 d3-0.5 d2-0.5 d2-0.5 d2-0.5 d2-0.5 d2-0.5 d2-0.5 d2-0.5 a2-0.5 d2-0.5 d2-0.5 d2-0.5 f3-0.5 d2-0.5 a2-0.5 d2-0.5 d2-0.5 d3-0.5 d2-0.5 d2-0.5 a2-0.5 c3-0.5 d2-0.5 d2-0.5 d2-0.5 d2-0.5 d2-0.5 e2-1.0 d2-0.5 d2-0.3333333333333333 d2-0.3333333333333333 d2-0.5 d2-0.5 d2-0.5 d2-0.5 d2-0.5 d3-0.5 a2-0.5 a2-0.5 d2-0.5 d2-0.5 d2-0.5 d2-0.5 d2-0.5 r-0.5 d2-0.5 d2-0.5 d2-0.5 d2-0.5 b2-1.0 d2-0.5 d2-0.3333333333333333 d2-0.5 d2-0.5 d2-0.5 d2-0.5 d2-0.5 d2-0.5 d2-0.5 d2-0.5 d2-0.5 a4-0.25 d2-0.5 d2-0.5 d2-0.5 d3-0.5 d2-0.5 e2-0.5 d2-0.3333333333333333 d2-0.5 d2-0.5 d2-0.5 d2-0.5 d2-0.5 d2-0.5 d2-0.5 d3-0.5 d2-0.5 d2-0.5 d2-0.5 a2-0.5 d2-0.5 d2-0.5 d3-1.0 a4-0.25 d2-0.5 d2-0.5 d2-0.5 d2-0.5 d3-0.5 d2-0.5 d2-0.5 d2-0.5 d2-0.5 d2-0.5 d2-0.5 d2-0.5 d3-0.25 d3-0.5 d2-0.5 d2-0.5 a2-0.5 f#3-0.5 d3-0.5 e2-0.5 a2-0.5 d2-0.5 a2-0.5 d2-0.5 d2-0.5 d2-0.5 e2-0.5 d2-0.5 d2-0.5 a2-0.5 d2-0.3333333333333333 d2-0.5 d2-0.5 d2-0.5 d3-0.5 d2-0.3333333333333333 d2-0.5 a2-0.5 b2-0.5 d2-0.5 d2-0.5 d2-0.5 d2-0.5 d2-0.5 d2-0.5 d2-0.5 d2-0.5 d2-0.5 d2-0.5 d2-0.5 d2-0.5 d2-0.5 d2-0.5 d2-0.5 d2-0.3333333333333333 d2-0.5 d2-0.5 d2-0.5 d2-0.5 d2-0.5 d2-0.5 d2-0.3333333333333333 d2-0.5 d2-0.5 d2-0.5 d2-0.5 d2-0.3333333333333333 e3-0.25 d2-0.5 d2-0.5 d2-0.5 d2-0.5 d2-0.5 d3-0.5 d2-0.5 d2-0.5 d2-0.5 d2-0.5 d2-0.5 r-1.0 d2-0.5 d2-0.5 d2-0.5 d2-0.5 a2-0.5 d3-0.5 d2-0.5 d2-0.5 d2-0.5 d2-0.5 d2-0.5 d3-1.0 d2-0.5 d2-0.5 d2-0.5 a2-0.5 d2-0.5 a2-0.5 d2-0.5 d2-0.5 d2-0.3333333333333333 a2-0.5 d2-0.5 d2-0.5 d2-0.5 d2-0.5 a2-0.5 d2-0.5 a2-0.5 d2-0.5 a2-0.5 d2-0.5 d2-0.5 d2-0.5 a2-0.5 e2-0.5 d2-0.5 d3-0.5 d2-0.5 d2-0.5 d2-0.5 a2-0.5 e3-0.5 d2-0.5 d2-0.5 d3-0.5 d2-0.5 d2-0.5 d2-0.5 a2-0.5 d2-0.5 a2-0.5 d2-0.5 d2-0.5 d2-0.5 e3-0.25 a2-0.5 d2-0.5 d2-0.5 d2-0.5 d2-0.5 d2-0.5 d2-0.5 d2-0.5 d2-0.3333333333333333 d3-0.5 e3-0.5 e2-0.25 e3-0.5 d2-0.5 d3-0.5 d2-0.5 d2-0.3333333333333333 e3-0.5 d2-0.5 c3-0.5 d2-0.5 d2-0.5 d2-0.5 d3-0.5 d2-0.5 d2-0.5 d3-0.5 d2-0.3333333333333333 d2-0.5 d2-0.5 d3-0.5 d2-0.5 e2-0.5 d2-0.5 a2-0.5 d3-0.25 d3-0.5 e3-0.5 a2-0.5 d2-0.5 a2-0.5 d2-0.5 d2-0.5 d2-0.5 a2-0.5 d2-0.5 r-1.0 d2-0.5 d2-0.5 d2-0.3333333333333333 a2-0.5 d2-0.5 d2-0.5 d2-0.5 e3-0.5 d2-0.5 d2-0.5 d2-0.5 d2-0.3333333333333333 d2-0.5 d2-0.5 d2-0.5 d2-0.5 d2-0.5 d2-0.5 d2-0.5 d2-0.5 d2-0.5 d2-0.5 d2-0.5 d2-0.5 d2-0.5 a2-0.5 d2-0.5 d2-0.5 d3-0.5 a2-0.5 d2-0.3333333333333333 d2-0.5 d2-0.5 d2-0.5 d2-0.5 d2-0.3333333333333333 d2-0.5 e3-0.5 a2-0.5 d2-0.5 a2-0.5 d2-0.5 d2-0.5 d2-0.5 d2-0.5 d2-0.5 d2-0.5 d2-0.5 d2-0.5 d2-0.5 d2-0.5 d2-0.5 a2-0.5 e2-0.5 d2-0.5 d2-0.5 d2-0.5 d2-0.3333333333333333 d3-0.5 d2-0.5 d2-0.5 d2-0.5 d2-0.5 d3-0.5 d2-0.5 d2-0.5 d2-0.5 d2-0.5 d2-0.5 d2-0.5 e3-0.5 d2-0.3333333333333333 d2-0.5 d2-0.5 d2-0.5 a2-0.5 d2-0.5 a2-0.5 d2-0.5 a2-0.5 a2-0.5 d2-0.3333333333333333 d2-0.3333333333333333 d3-0.5 d3-0.5 d2-0.3333333333333333 d2-0.5 d2-0.5 d2-0.5 a2-0.5 d3-0.5 d3-0.5 d2-0.5 d2-0.5 d2-0.5 d2-0.3333333333333333 d3-0.5 a2-0.5 d3-0.5 d2-0.5 d2-0.5 d2-0.5 d2-0.3333333333333333 d2-0.5 d2-0.5 d2-0.5 d2-0.5 d2-0.5 a2-0.5 d2-0.5 d2-0.5 d2-0.5 d3-0.25 r-0.5 d2-0.5 d3-0.5 a2-0.5 d2-0.3333333333333333 d2-0.5 d2-0.3333333333333333 d2-0.5 d2-0.5 d2-0.5 d2-0.5 d2-0.5 d2-0.5 a2-0.5 d2-0.5 d2-0.5 d2-0.5 d3-0.5 d3-0.5 a2-0.5 d2-0.5 d2-0.5 d2-0.5 d2-0.5 d2-0.3333333333333333 d2-0.5 d2-0.5 r-0.5 d3-0.5 d2-0.5 d2-0.5 d2-0.5 d2-0.3333333333333333 d2-0.5 d2-0.5 d3-0.5 d2-0.5 d2-0.5 d3-0.5 d3-0.5 d2-0.5 d2-0.5 d2-0.5 d2-0.5 d2-0.5 d2-0.5 d2-0.5 d2-0.5 d2-0.5 d2-0.5 d2-0.5 a2-0.5 d2-0.5 a2-0.5 d2-0.5 d3-0.5 d2-0.5 d2-0.5 a2-0.5 a2-0.5 a2-0.25 d2-0.25 d2-0.5 d2-0.5 d2-0.5 d2-0.5 d2-0.3333333333333333 d2-0.5 d2-0.5'



notes = string.split(' ')

s = music21.stream.Stream()
for row in notes:
    data = row.split("-")
    note = data[0]
    duration= float(data[1])
    
    if not note == 'x':
        if note == 'r':
            s.append(music21.note.Rest(quarterLength=duration))
        else:
            s.append(music21.note.Note(note, quarterLength=duration))


print(s)
s.write("midi", "output.mid")

In [ ]:
print(history.history)